In [7]:
%pip install xgboost 
%pip install tslearn
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

#from sktime.datasets import load_UCR_UEA_dataset
from tslearn.datasets import UCR_UEA_datasets

#1d interpolation
from scipy.interpolate import interp1d

In [22]:
start_date ='2014-12-31'
end_date = '2021-05-31'
price = yf.download("BTC-USD", start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [23]:
price = price.reset_index()

In [24]:
price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-12-31,310.914001,320.192993,310.210999,320.192993,320.192993,13942900
1,2015-01-01,320.434998,320.434998,314.002991,314.248993,314.248993,8036550
2,2015-01-02,314.079010,315.838989,313.565002,315.032013,315.032013,7860650
3,2015-01-03,314.846008,315.149994,281.082001,281.082001,281.082001,33054400
4,2015-01-04,281.145996,287.230011,257.612000,264.195007,264.195007,55629100


In [25]:
# Due to the high volatility nature of crypto currency we chose a 1 day lag
lag = 1  # Number of lags
price['Close Lagged Price'] = price['Close'].shift(lag)

In [26]:
# We will compute 7, 14, 21 simple moving average moving average 

price['sma7'] = price['Close'].rolling(window=7).mean()
price['sma14'] = price['Close'].rolling(window=14).mean()
price['sma21'] = price['Close'].rolling(window=21).mean()


price['ema12'] = price['Close'].ewm(span=12, adjust=False).mean()
price['ema26'] = price['Close'].ewm(span=26, adjust=False).mean()

# Calculate MACD Line
price['MACD line'] = price['ema12'] - price['ema26']

# Calculate Signal Line
price['Signal line'] = price['MACD line'].ewm(span=9, adjust=False).mean()

# Calculate MACD Histogram
price['MACD histogram'] = price['MACD line'] - price['Signal line']


In [27]:
price_2= price.Close.to_frame()
price['PriceChange'] = price['Close'].pct_change() * 100 
price['Label'] = np.where(price['PriceChange'] > 0, 1, 0)

price['change']=price_2.apply(lambda x: x/x.shift(1)-1)
price['log_change']=price_2.apply(lambda x: np.log(x)-np.log(x.shift(1)))
price['change_label']=price['change'].apply (lambda x: x>0.02)

In [28]:
window = 14  # Number of periods for RSI calculation

# Calculate price changes
delta = price['Close'].diff()

# Calculate gains and losses
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

# Calculate average gains and losses
avg_gain = gain.rolling(window=window).mean()
avg_loss = loss.rolling(window=window).mean()

# Calculate relative strength (RS) and RSI
rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))

price['rsi'] = rsi


In [29]:
price = price.dropna()

In [33]:
X = price.drop(['Date', 'change', 'log_change', 'change_label','Label'], axis=1)
Y = price['Label']

In [34]:
X = X.to_numpy()

In [37]:
Y = Y.to_numpy()

In [38]:
Y.shape

(2323,)

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
# Split the data into training, validation, and testing sets
# First, split into training and temp sets (80% training, 20% temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
# Then, split the temp set into validation and testing sets (50% validation, 50% testing)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [41]:
#encode the labels
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [46]:
#interpolate data
length = 100
def interpolate(X, length):
    X_interpolated = []
    for i in range(X.shape):
        x = np.linspace(0, 1, X.shape[1])
        f = interp1d(x, X[i], axis=0)
        xnew = np.linspace(0, 1, length)
        ynew = f(xnew)
        X_interpolated.append(ynew)
    return np.array(X_interpolated)

In [47]:
X_train = interpolate(X_train, length)
X_test = interpolate(X_test, length)

TypeError: 'tuple' object cannot be interpreted as an integer

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Sample financial time series data (replace with your own data)
# For demonstration, we'll use a simple sine wave pattern.
data = np.sin(np.linspace(0, 100, 1000)) + np.random.randn(1000) * 0.1

# Convert data to a DataFrame
df = pd.DataFrame({'Price': data})

# Normalize the data (scaling between 0 and 1)
scaler = MinMaxScaler()
df['Price'] = scaler.fit_transform(df[['Price']])

# Prepare the data for LSTM training
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        sequences.append((seq, target))
    return np.array([x[0] for x in sequences]), np.array([x[1] for x in sequences])

sequence_length = 10  # Adjust as needed
X, y = create_sequences(df['Price'].values, sequence_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create and train the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(sequence_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=32)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values to the original scale
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Visualize the predictions vs. actual values
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title('Financial Time Series Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.show()


Epoch 1/50
25/25 [==============================] - 1s 3ms/step - loss: 0.1066
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0240
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0202
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 0.0179
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 0.0159
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0136
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 0.0108
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 9/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 10/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 11/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 12/50
25/25 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 13/50
25/25 [==============================

ValueError: Expected 2D array, got 1D array instead:
array=[0.07911889 0.14244136 0.17171454 0.10595537 0.136661   0.20781518
 0.18567156 0.17640327 0.14780535 0.29324373 0.30279806 0.38998222
 0.38525454 0.41321143 0.49661165 0.53551645 0.50967679 0.63288079
 0.56083052 0.6452561  0.65716486 0.78694508 0.77670828 0.81431973
 0.78493574 0.88419999 0.88170942 0.89600976 0.84540848 0.87205763
 0.88965944 0.80905509 0.84179954 0.84170135 0.84243194 0.7593448
 0.78224348 0.68935738 0.70422299 0.71481863 0.65540209 0.67101191
 0.58358702 0.63759777 0.52656025 0.53832316 0.42134252 0.42898913
 0.4130653  0.32877633 0.30103111 0.3136112  0.2066688  0.21670733
 0.22716462 0.21052353 0.16413549 0.11204028 0.09174841 0.12735396
 0.14143271 0.01101004 0.04780975 0.1339863  0.12381062 0.08962675
 0.12542405 0.16178029 0.12105077 0.15644264 0.23520565 0.26181615
 0.25986006 0.36378119 0.42133933 0.4282586  0.49374367 0.50047688
 0.59633267 0.57745901 0.59730504 0.6985511  0.6922828  0.68757862
 0.7

In [49]:
X

array([[0.44870943, 0.56228347, 0.52640853, ..., 0.70843262, 0.75519158,
        0.81185503],
       [0.56228347, 0.52640853, 0.63804732, ..., 0.75519158, 0.81185503,
        0.79914562],
       [0.52640853, 0.63804732, 0.61941695, ..., 0.81185503, 0.79914562,
        0.87909337],
       ...,
       [0.06748078, 0.11263845, 0.08940655, ..., 0.10866882, 0.19169427,
        0.17564377],
       [0.11263845, 0.08940655, 0.08906414, ..., 0.19169427, 0.17564377,
        0.27088307],
       [0.08940655, 0.08906414, 0.0491815 , ..., 0.17564377, 0.27088307,
        0.24690841]])